<a href="https://colab.research.google.com/github/nick01as/Gazer/blob/main/GetLMFilesCompile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Import LM Data
LM2021 = pd.read_csv("LM2021.csv")
LM2020 = pd.read_csv("LM2020.csv")
LM2019 = pd.read_csv("LM2019.csv")
LM2018 = pd.read_csv("LM2018.csv")
LM2017 = pd.read_csv("LM2017.csv")
LM2016 = pd.read_csv("LM2016.csv")
LM2015 = pd.read_csv("LM2015.csv")

LM2021 = LM2021.drop(columns = ['Unnamed: 0'])
LM2020 = LM2020.drop(columns = ['Unnamed: 0'])
LM2019 = LM2019.drop(columns = ['Unnamed: 0'])
LM2018 = LM2018.drop(columns = ['Unnamed: 0'])
LM2017 = LM2017.drop(columns = ['Unnamed: 0'])
LM2016 = LM2016.drop(columns = ['Unnamed: 0'])
LM2015 = LM2015.drop(columns = ['Unnamed: 0'])


# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['lat','lng'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]


# Find City

def find_city(lat,lng):
  # print (lat,lng)
  lowLB = 0
  highLB = len(CityCoor.index) 
  lowUB = 0
  highUB = len(CityCoor.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if CityCoor.iloc[mid,0] < lat - 0.12:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if CityCoor.iloc[mid,0] <= lat + 0.12:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB

  # print("Lower Bound Coordinate", CityCoor.iloc[lowLB,0], CityCoor.iloc[highLB,0])
  # print("Upper Bound Coordinate", CityCoor.iloc[lowUB,0], CityCoor.iloc[highUB,0])
  best = 2147483647
  city = ""
  country = ""

  for idx in range (lowLB, lowUB):
    dif = dist(lat,lng,CityCoor.iloc[idx,0],CityCoor.iloc[idx,1])
    if dif < best:
      city = CityCoor.iloc[idx,2]
      country = CityCoor.iloc[idx,3]
      best = dif

  return (city,country)

cityList = []
errorList = []
error_count = 0

LM2017

#--------------------------------------------------------------------------------------------------------------------------------------------
for idx in range (len(LM2019.index)):
  print (idx)
  locInfo = find_city(LM2019.iloc[idx,0],LM2019.iloc[idx,1])
  if locInfo[1] != LM2019.iloc[idx,5] and (locInfo[1][:8] != 'United S' and locInfo[1][:8] != 'Netherla' and 
                                           locInfo[1][:8] != 'Falkland' and locInfo[1][:7] != 'Czechia' and locInfo[1] != 'Mexico'
                                           and locInfo[1] != 'Macedonia' and locInfo[1] != 'Slovenia' and locInfo[1] != 'Trinidad and Tobago'
                                           and locInfo[1] != 'Belgium' and locInfo[1] != 'Vatican City' and locInfo[1] != 'Vatican' and locInfo[1] != 'Argentina' 
                                           and locInfo[1] != 'West Bank' and locInfo[1] != 'Uruguay' and locInfo[1] != 'Thailand' and locInfo[1] != 'Bolivia' 
                                           and locInfo[1] != 'France' and locInfo[1] != 'Kuwait' and locInfo[1] != 'Brazil' and locInfo[1] != 'Slovakia' and locInfo[1] != 'Hungary'
                                           and locInfo[1] != 'Finland' and locInfo[1] != 'Bosnia and Herzegovina' and locInfo[1] != ''):
    print ("Error: Wrong Country",locInfo[1],LM2019.iloc[idx,5])
    errorList.append(1)
    error_count = error_count + 1
  elif locInfo[0] == '':
    errorList.append(1)
  else:
    errorList.append(0)

  cityList.append(locInfo[0])


print("")
print(cityList)
print(error_count)
LM2019City = LM2019.drop(columns = ['Latitude','Longitude'])
LM2019City['City'] = cityList
LM2019City['Error'] = errorList
cols = list(LM2019City.columns.values)
LM2019City = LM2019City[cols[0:3] + [cols[-2]] + [cols[-3]] + [cols[-1]]]

idx = 0
while idx < len(LM2019City.index):
  if LM2019City.iloc[idx,5] == 1:
    LM2019City = LM2019City.drop(index = LM2019City.index[idx])
  else:
    idx = idx + 1

LM2019City

LM2019City.to_csv('LM2019City.csv')
files.download('LM2019City.csv')

Streaming output truncated to the last 5000 lines.
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
Error: Wrong Country Switzerland Austria
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
Error: Wrong Country Switzerland Austria
591
592
Error: Wrong Country Switzerland Austria
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
Error: Wrong Country Switzerland Austria
635
Error: Wrong Country Switzerland Austria
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Import LM Data
LM2021 = pd.read_csv("LM2021.csv")
LM2020 = pd.read_csv("LM2020.csv")
LM2019 = pd.read_csv("LM2019.csv")
LM2018 = pd.read_csv("LM2018.csv")
LM2017 = pd.read_csv("LM2017.csv")
LM2016 = pd.read_csv("LM2016.csv")
LM2015 = pd.read_csv("LM2015.csv")

LM2021 = LM2021.drop(columns = ['Unnamed: 0'])
LM2020 = LM2020.drop(columns = ['Unnamed: 0'])
LM2019 = LM2019.drop(columns = ['Unnamed: 0'])
LM2018 = LM2018.drop(columns = ['Unnamed: 0'])
LM2017 = LM2017.drop(columns = ['Unnamed: 0'])
LM2016 = LM2016.drop(columns = ['Unnamed: 0'])
LM2015 = LM2015.drop(columns = ['Unnamed: 0'])


# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['lat','lng'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]


# Find City

def find_city(lat,lng):
  # print (lat,lng)
  lowLB = 0
  highLB = len(CityCoor.index) 
  lowUB = 0
  highUB = len(CityCoor.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if CityCoor.iloc[mid,0] < lat - 0.12:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if CityCoor.iloc[mid,0] <= lat + 0.12:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB

  # print("Lower Bound Coordinate", CityCoor.iloc[lowLB,0], CityCoor.iloc[highLB,0])
  # print("Upper Bound Coordinate", CityCoor.iloc[lowUB,0], CityCoor.iloc[highUB,0])
  best = 2147483647
  city = ""
  country = ""

  for idx in range (lowLB, lowUB):
    dif = dist(lat,lng,CityCoor.iloc[idx,0],CityCoor.iloc[idx,1])
    if dif < best:
      city = CityCoor.iloc[idx,2]
      country = CityCoor.iloc[idx,3]
      best = dif

  return (city,country)

cityList = []
errorList = []
error_count = 0

LM2017

#--------------------------------------------------------------------------------------------------------------------------------------------
for idx in range (len(LM2020.index)):
  print (idx)
  locInfo = find_city(LM2020.iloc[idx,0],LM2020.iloc[idx,1])
  if locInfo[1] != LM2020.iloc[idx,5] and (locInfo[1][:8] != 'United S' and locInfo[1][:8] != 'Netherla' and 
                                           locInfo[1][:8] != 'Falkland' and locInfo[1][:7] != 'Czechia' and locInfo[1] != 'Mexico'
                                           and locInfo[1] != 'Macedonia' and locInfo[1] != 'Slovenia' and locInfo[1] != 'Trinidad and Tobago'
                                           and locInfo[1] != 'Belgium' and locInfo[1] != 'Vatican City' and locInfo[1] != 'Vatican' and locInfo[1] != 'Argentina' 
                                           and locInfo[1] != 'West Bank' and locInfo[1] != 'Uruguay' and locInfo[1] != 'Thailand' and locInfo[1] != 'Bolivia' 
                                           and locInfo[1] != 'France' and locInfo[1] != 'Kuwait' and locInfo[1] != 'Brazil' and locInfo[1] != 'Slovakia' and locInfo[1] != 'Hungary'
                                           and locInfo[1] != 'Finland' and locInfo[1] != 'Bosnia and Herzegovina' and locInfo[1] != ''):
    print ("Error: Wrong Country",locInfo[1],LM2020.iloc[idx,5])
    errorList.append(1)
    error_count = error_count + 1
  elif locInfo[0] == '':
    errorList.append(1)
  else:
    errorList.append(0)

  cityList.append(locInfo[0])

print("")
print(cityList)
print(error_count)
LM2020City = LM2020.drop(columns = ['Latitude','Longitude'])
LM2020City['City'] = cityList
LM2020City['Error'] = errorList
cols = list(LM2020City.columns.values)
LM2020City = LM2020City[cols[0:3] + [cols[-2]] + [cols[-3]] + [cols[-1]]]

idx = 0
while idx < len(LM2020City.index):
  if LM2020City.iloc[idx,5] == 1:
    LM2020City = LM2020City.drop(index = LM2020City.index[idx])
  else:
    idx = idx + 1

LM2020City

LM2020City.to_csv('LM2020City.csv')
files.download('LM2020City.csv')

Streaming output truncated to the last 5000 lines.
8033
8034
8035
8036
8037
8038
8039
8040
8041
8042
8043
8044
8045
8046
8047
8048
8049
8050
8051
8052
8053
8054
8055
8056
8057
8058
8059
8060
8061
8062
8063
8064
8065
8066
8067
8068
8069
8070
8071
8072
8073
8074
8075
8076
8077
8078
8079
8080
8081
8082
8083
8084
8085
8086
Error: Wrong Country Russia Australia
8087
8088
8089
8090
8091
8092
8093
8094
8095
8096
8097
8098
8099
8100
8101
8102
8103
8104
8105
8106
8107
8108
8109
8110
8111
8112
8113
8114
8115
8116
8117
8118
8119
8120
8121
8122
8123
8124
8125
8126
8127
8128
8129
8130
8131
8132
8133
8134
8135
8136
8137
8138
8139
8140
8141
8142
8143
8144
8145
8146
8147
8148
8149
8150
8151
8152
8153
8154
8155
8156
8157
8158
8159
8160
8161
8162
8163
8164
8165
8166
8167
8168
8169
8170
8171
8172
8173
8174
8175
8176
8177
8178
8179
8180
8181
8182
8183
8184
8185
8186
8187
8188
8189
8190
8191
8192
8193
8194
8195
8196
8197
8198
8199
8200
8201
8202
8203
8204
8205
8206
8207
8208
8209
8210
8211
8212
8213
8214
8

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>